# __Handling multiple sequences (PyTorch)__

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## Модели ожидают пакет входных данных

Последовательности преобразуются в списки чисел. Давайте преобразуем этот список чисел в тензор и отправим его в модель:

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

In [4]:
print(tokens)
print(ids)
print(input_ids)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
         2026,  2878,  2166,  1012])


In [ ]:
# This line will fail.
model(input_ids)

## Ошибка!!!  
Проблема в том, что мы отправили в модель одну последовательность, тогда как 🤗 Модели Transformers по умолчанию ожидают несколько предложений. Здесь мы попытались сделать все, что токенизатор делал за кулисами, когда мы применяли его к последовательности. Но если вы посмотрите внимательно, вы увидите, что токенизатор не просто преобразовал список идентификаторов входных данных в тензор, он добавил измерение поверх него:

Пакетирование — это процесс отправки нескольких предложений через модель, все сразу. Если у вас есть только одно предложение, вы можете просто создать пакет с одной последовательностью:

In [5]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


Попробуем снова с новой размерностью

In [6]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# sequence = "I've been waiting for a HuggingFace course my whole life."

# tokens = tokenizer.tokenize(sequence)
# ids = tokenizer.convert_tokens_to_ids(tokens)

# _________Было_______________
# input_ids = torch.tensor(ids)
# _________Стало_______________
input_ids = torch.tensor([ids])

print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


Пакетирование — это процесс отправки нескольких предложений через модель, все сразу. Если у вас есть только одно предложение, вы можете просто создать пакет с одной последовательностью:

In [9]:
# This is a batch of two identical sequences!
batched_ids = [ids, ids]
print(batched_ids)

[[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012], [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]]


In [13]:
input_ids_2 = torch.tensor(batched_ids)
output_2 = model(input_ids_2)
print("Logits:", output_2.logits)

Logits: tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


Пакетирование позволяет модели работать, когда вы передаете ей несколько предложений.

### Заполнение входных данных

In [14]:
# Следующий список списков не может быть преобразован в тензор:
batched_ids = [
[200, 200, 200],
[200, 200]
]

Чтобы обойти это, мы используем заполнение, чтобы сделать наши тензоры прямоугольными. Заполнение гарантирует, что все наши предложения будут иметь одинаковую длину, добавляя специальное слово, называемое токеном заполнения, к предложениям с меньшим количеством значений. В нашем примере результирующий тензор выглядит так:

In [15]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

Идентификатор маркера заполнения можно найти в `tokenizer.pad_token_id`. Давайте используем его и отправим наши два предложения через модель по отдельности и в пакетном режиме:

In [16]:
sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


<font color="lightblue">Что-то не так с логитами в наших пакетных прогнозах: вторая строка должна быть такой же, как логиты для второго предложения, но у нас совершенно разные значения!

Это потому, что ключевой особенностью моделей Transformer являются слои внимания, которые контекстуализируют каждый токен. Они будут учитывать токены заполнения, поскольку они обслуживают все токены последовательности. Чтобы получить тот же результат при передаче отдельных предложений разной длины через модель или при передаче пакета с теми же предложениями и примененным заполнением, нам нужно указать этим слоям внимания игнорировать токены заполнения. Это делается с помощью маски внимания.</font>

## Attention masks

Маски внимания — это тензоры с точно такой же формой, как и тензор входных идентификаторов, заполненные нулями и единицами: единицы указывают, что соответствующие токены должны быть учтены, а нули указывают, что соответствующие токены не должны быть учтены (т. е. они должны игнорироваться слоями внимания модели).

Давайте завершим предыдущий пример с маской внимания:

In [17]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


## Longer sequences (Более длинные последовательности)

Более длинные последовательности
В моделях Transformer существует ограничение на длину последовательностей, которые мы можем передать моделям. Большинство моделей обрабатывают последовательности длиной до `512` или `1024` токенов и будут аварийно завершать работу при запросе на обработку более длинных последовательностей. Есть два решения этой проблемы:

Используйте модель с более длинной поддерживаемой длиной последовательности.
Усекайте последовательности.
Модели имеют разную поддерживаемую длину последовательностей, и некоторые специализируются на обработке очень длинных последовательностей.  
- Longformer — один из примеров, а другой
— LED. Если вы работаете над задачей, требующей очень длинных последовательностей, мы рекомендуем вам взглянуть на эти модели.

В противном случае мы рекомендуем вам усекать последовательности, указав параметр max_sequence_length:

In [ ]:
sequence = sequence[:max_sequence_length]